# Lab-05: Visualizing tweets from the 2020 US presidential election

In [6]:
import os
import pandas as pd

# Dataset is a randomly sampled subset of: https://www.kaggle.com/manchunhui/us-election-2020-tweets
trump = pd.read_csv("2020_tweets_trump.csv", lineterminator='\n')
biden = pd.read_csv("2020_tweets_biden.csv", lineterminator='\n')

In [7]:
len(biden), len(trump)

(100000, 100000)

In [8]:
M = 10000
trump = trump.sample(n=M//2)
biden = biden.sample(n=M//2)

In [9]:
biden_tweets = biden['tweet'].tolist()
trump_tweets = trump['tweet'].tolist()

In [10]:
biden_tweets[3023]

'#CNN u ekstazi. #Biden'

In [11]:
type(biden_tweets)

list

# Preprocessing pipeline

In [12]:
import re
from tqdm import tqdm
from typing import List

import spacy
from spacy.language import Language

pipeline_name = '2020ElectionTweets'


def camel_case_split(str):
    """ This function turns in #Biden2020 into Biden 2020 """
    return " ".join([wrd for wrd in re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)])


@Language.component(pipeline_name)
def preprocess(doc):
    doc = [token for token in doc if not token.is_punct]
    # doc = [token for token in doc if not token.is_stop]
    doc = [token.text.lower().strip() for token in doc]
    doc = [token for token in doc if 0 < len(token) <= 12]
    return " ".join(doc)


class Pipeline:
    
    # http://emailregex.com/
    email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)
    *|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]
    |\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9]
    (?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}
    (?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:
    (?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""
    # replace = [ (pattern-to-replace, replacement),  ...]
    replace = [
        ("<[^>]*>", " "),
        (email_re, " "),                           # Matches emails
        (r"(?<=\d),(?=\d)", ""),                   # Remove commas in numbers
        (r"\d+", " "),                             # Map digits to special token <numbr>
        (r"[*\^\.$&@<>,\-/+{|}=?#:;'\"\[\]]", ""), # Punctuation and other junk
        (r"[\n\t\r]", " "),                        # Removes newlines, tabs, creturn
        (r"[^\x00-\x7F]+", ""),                    # Removes non-ascii chars
        (r"\\+", " "),                             # Removes double-backslashs
        (r"\s+n\s+", " "),                         # 'n' leftover from \\n
        (r"\s+", " ")                              # Strips extra whitespace
    ]
    
    def __init__(self):
        self.pipeline = spacy.load('en_core_web_sm')
        self.pipeline.add_pipe(pipeline_name);
        
    def __call__(self, *args, **kwargs):
        return self.transform(*args, **kwargs)

    def transform(self, doc: str):
        for repl in self.replace:
            doc = re.sub(repl[0], repl[1], doc)
        doc = camel_case_split(doc)
        return self.pipeline(doc)
    
pipeline = Pipeline();

In [13]:
from tqdm import tqdm

with tqdm(total=M//2) as bar:
    for i, (bt, tt) in enumerate(zip(biden_tweets, trump_tweets)):
        biden_tweets[i] = pipeline(bt)
        trump_tweets[i] = pipeline(tt)
        bar.update()

100%|██████████| 5000/5000 [00:41<00:00, 120.91it/s]


In [37]:
biden_tweets[3021]

'us election results joe biden us elections on ab us elections ym vo ev'

### Concatenate documents for vocab generation

In [15]:
all_tweets = biden_tweets + trump_tweets

## (20 pts) Task I: Train a Doc2Vec model (using the Gensim package) on tweets from the 2020 US presidential election

*Docs*: 

* https://radimrehurek.com/gensim/models/doc2vec.html

*Useful tutorials*: 

* https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial 
* https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

### Hyperparameters

In [16]:
# Change as needed
K = 20
word_frequency_threshold = 2
epochs = 10
lr = 0.01

In [43]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

documents = [TaggedDocument(all_tweets, [i]) for i, all_tweets in enumerate(common_texts)]
model = Doc2Vec(documents,vector_size=K, min_count=word_frequency_threshold, epochs=epochs)

In [41]:
sentences = [doc2vec.TaggedDocument(sentence, 'tag') for sentence in titlelist]

NameError: name 'titlelist' is not defined

In [39]:
tweets_split = [row.split() for row in all_tweets]

In [45]:
# Your code goes here
#Build vocabulary
model.build_vocab(documents)

In [46]:
model.train(documents,total_examples=model.corpus_count, epochs=epochs)

In [47]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[ 0.01795956  0.01144458 -0.0233179  -0.00772732 -0.00205748  0.02062779
  0.00868887 -0.00773061 -0.00015735 -0.01275906  0.01478313  0.00916422
  0.01442062 -0.0109679  -0.0140332   0.01916066 -0.00834236 -0.01888616
  0.01555071  0.00806334]


## (10  pts) Task II: Evaluate your model by computing the most similar documents (tweets) to new (perhaps made up) tweets

In [48]:
# Template function
def find_similar_tweets(tweet, top_n=10):
    doc_vector = model.infer_vector(tweet)
    sims = model.dv.most_similar([doc_vector], topn=top_n)
    return sims

In [50]:
# Your code goes here
find_similar_tweets(['Trump', 'is', 'an', 'asshole'])

[(2, 0.30320504307746887),
 (0, 0.24533195793628693),
 (3, 0.2099137157201767),
 (7, 0.10132831335067749),
 (6, 0.08364967256784439),
 (5, 0.022875366732478142),
 (1, -0.015743907541036606),
 (4, -0.05227864533662796),
 (8, -0.3419002592563629)]

## (10 pts extra credit) Task III: Produce a scatter plot of the compressed document embeddings (2D or 3D)

*Useful resources*:

* http://projector.tensorflow.org/

In [ ]:
# Your code goes here